In [19]:
import pandas as pd
import numpy as np

import requests
import datetime

import yfinance as yf

from alpaca.trading.client import TradingClient
from alpaca.trading.enums import OrderSide, TimeInForce
from alpaca.trading.requests import MarketOrderRequest, StopLossRequest
from alpaca.trading.stream import TradingStream

import config
# from reversal_bot import Reversal
# from portfolio import Portfolio

In [3]:
client = TradingClient(api_key = config.API_KEY, secret_key=config.SECRET_KEY, paper=True)
account = dict(client.get_account())
for i,j in account.items():
    print(i, j)

id 41abbed0-4bcc-4e17-ad2b-327e9246ae36
account_number PA3VU3Y41XE7
status AccountStatus.ACTIVE
crypto_status AccountStatus.ACTIVE
currency USD
buying_power 397870.8
regt_buying_power 198935.4
daytrading_buying_power 397870.8
non_marginable_buying_power 99467.7
cash 99467.7
accrued_fees 0
pending_transfer_out None
pending_transfer_in 0
portfolio_value 99467.7
pattern_day_trader True
trading_blocked False
transfers_blocked False
account_blocked False
created_at 2023-08-04 15:05:19.996002+00:00
trade_suspended_by_user False
multiplier 4
shorting_enabled True
equity 99467.7
last_equity 99467.7
long_market_value 0
short_market_value 0
initial_margin 0
maintenance_margin 0
last_maintenance_margin 0
sma 99467.7
daytrade_count 15


In [5]:
client.list_positions()

AttributeError: 'TradingClient' object has no attribute 'list_positions'

In [9]:
for i in dir(client):
    print(i)

__abstractmethods__
__class__
__delattr__
__dict__
__dir__
__doc__
__eq__
__format__
__ge__
__getattribute__
__gt__
__hash__
__init__
__init_subclass__
__le__
__lt__
__module__
__ne__
__new__
__reduce__
__reduce_ex__
__repr__
__setattr__
__sizeof__
__slots__
__str__
__subclasshook__
__weakref__
_abc_impl
_api_key
_api_version
_base_url
_get_auth_headers
_get_default_headers
_oauth_token
_one_request
_request
_retry
_retry_codes
_retry_wait
_return_paginated_result
_sandbox
_secret_key
_session
_use_basic_auth
_use_raw_data
_validate_credentials
_validate_pagination
add_asset_to_watchlist_by_id
cancel_order_by_id
cancel_orders
close_all_positions
close_position
create_watchlist
delete
delete_watchlist_by_id
get
get_account
get_account_configurations
get_all_assets
get_all_positions
get_asset
get_calendar
get_clock
get_corporate_announcement_by_id
get_corporate_announcements
get_open_position
get_order_by_client_id
get_order_by_id
get_orders
get_watchlist_by_id
get_watchlists
patch
post


In [11]:
print(client.get_all_positions())

[]


In [12]:
tickers = yf.Tickers('^IXIC')
print(tickers)

yfinance.Tickers object <^IXIC>


In [14]:
for i in tickers.tickers:
    print(i)

^IXIC


In [20]:
table = pd.read_html('https://stockmarketmba.com/stocksinthenasdaq100.php')

In [21]:
print(table)

[                                                   0
0  Due to health issues, this site is no longer m...,    Symbol                       Description      Category2  Category3  \
0     CEG  Constellation Energy Corporation  Common stocks  Large cap   
1    RIVN   Rivian Automotive, Inc. Class A  Common stocks  Large cap   
2    META              Meta Platforms, Inc.  Common stocks  Large cap   
3    ABNB                      Airbnb, Inc.  Common stocks  Large cap   
4    LCID                 Lucid Group, Inc.  Common stocks  Large cap   
..    ...                               ...            ...        ...   
95   MCHP          Microchip Technology Inc  Common stocks  Large cap   
96   ODFL     Old Dominion Freight Line Inc  Common stocks  Large cap   
97    BKR      Baker Hughes Company Class A  Common stocks  Large cap   
98   FISV                        Fiserv Inc  Common stocks  Large cap   
99    CSX                          CSX Corp  Common stocks  Large cap   

               

In [24]:
print(table[1]['Symbol'])

0      CEG
1     RIVN
2     META
3     ABNB
4     LCID
      ... 
95    MCHP
96    ODFL
97     BKR
98    FISV
99     CSX
Name: Symbol, Length: 100, dtype: object
